<a style="float:left;" href="https://colab.research.google.com/github/ClaudeCoulombe/VIARENA/blob/master/Labos/Lab-MNIST/Identification_ChiffresManuscrits_MNIST-ResConv.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
<br/>
### Rappel - Fonctionnement d'un carnet web iPython

* Pour exécuter le code contenu dans une cellule d'un carnet iPython, cliquez dans la cellule et faites (⇧↵, shift-enter);
* Le code d'un carnet iPython s'exécute séquentiellement de haut en bas de la page. Souvent, l'importation d'une bibliothèque Python ou l'initialisation d'une variable est préalable à l'exécution d'une cellule située plus bas. Il est donc recommandé d'exécuter les cellules en séquence. Enfin, méfiez-vous des retours en arrière qui peuvent réinitialiser certaines variables;
* Pour obtenir de l'information sur une fonction, utilisez la commande Python `help(`"nom de la fonction"`)`

# Identification de chiffres manuscrits - jeu de données MNIST
## Réseau convolutif - LeNet

#### Inspiration:

Richmond Alake - <a href="https://towardsdatascience.com/understanding-and-implementing-lenet-5-cnn-architecture-deep-learning-a2d531ebc342" target='_blank'>Understanding and Implementing LeNet-5 CNN Architecture</a> - 25 juin 2020

Yann LeCun, Léon Bottou, Yoshua Bengio et Patrick Haffner <a href="http://yann.lecun.com/exdb/publis/pdf/lecun-98.pdf" target='_blank'>Gradient-Based Learning Applied to Document Recognition</a> - 1998



## Importation des bibliothèques Python

In [ ]:
import os
import matplotlib.pyplot as plt

import tensorflow as tf
print("TensorFlow version:",tf.__version__)
import keras
print("Keras version:",keras.__version__)

## Fixer le hasard pour la reproductibilité

La mise au point de réseaux de neurones implique certains processus aléatoires. Afin de pouvoir reproduire et comparer vos résultats d'expérience, vous fixez temporairement l'état aléatoire grâce à un germe aléatoire unique.

Pendant la mise au point, vous fixez temporairement l'état aléatoire pour la reproductibilité mais vous répétez l'expérience avec différents germes ou états aléatoires et prenez la moyenne des résultats.
<br/>
##### **Note**: Pour un système en production, vous ravivez simplement l'état  purement aléatoire avec l'instruction `GERME_ALEATOIRE = None`

In [ ]:
import os

# Définir un germe aléatoire
GERME_ALEATOIRE = 21

# Définir un état aléatoire pour Python
os.environ['PYTHONHASHSEED'] = str(GERME_ALEATOIRE)

# Définir un état aléatoire pour Python random
import random
random.seed(GERME_ALEATOIRE)

# Définir un état aléatoire pour NumPy
import numpy as np
np.random.seed(GERME_ALEATOIRE)

# Définir un état aléatoire pour TensorFlow
import tensorflow as tf
tf.random.set_seed(GERME_ALEATOIRE)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

print("Germe aléatoire fixé")

## Jeu de données - chiffres manuscrits MNIST

Le jeu de données MNIST (Modified National Institute of Standards and Technology) comporte 60,000 images en tons de gris de 28×28 pixels de chiffres manuscrits étiquetés de 0 à 9. Site web: http://yann.lecun.com/exdb/mnist/

Il est incorporé dans keras.datasets

### Lecture des données
### Séparation entre jeu de données d'entraînement et jeux de données de test

In [ ]:
# le jeu de données MNIST
from keras.datasets import mnist

dic_noms_classe = {
    0 : "0",
    1 : "1",
    2 : "2",
    3 : "3",
    4 : "4",
    5 : "5",
    6 : "6",
    7 : "7",
    8 : "8",
    9 : "9",
}

# lire le jeu de données MNIST et le diviser entre
# les données d'entrainement et les données de test
# MNIST est déjà divisé en un jeu de données d'entraînement (les 60 000 premières images)
# et un jeu de données de test (les 10 000 dernières images).
(attributs_entrainement, classes_cibles_entrainement), (attributs_test, classes_cibles_test) = mnist.load_data()


## Exploration des données

### Portrait des données

In [ ]:
# Portrait des données
print()
print('Entraînement: attributs=%s, classes=%s' % (attributs_entrainement.shape, classes_cibles_entrainement.shape))
print('Test: attributs=%s, classes=%s' % (attributs_test.shape, classes_cibles_test.shape))

In [ ]:
attributs_entrainement.shape

### Visualisation de données

In [ ]:
# Afficher les 24 premières images
print()
print("Quelques images avec leur étiquette de classe-cible...")
%matplotlib inline
# définir subplot
fig, axes = plt.subplots(nrows=4,ncols=6,figsize=(13,10))
for i_rangee in range(0,4):
    for i_colonne in range(0,6):
        axes[i_rangee,i_colonne].set_title(dic_noms_classe[int(classes_cibles_entrainement[i_rangee*6+i_colonne])],
                                           fontsize=10, color='red')
        axes[i_rangee,i_colonne].imshow(attributs_entrainement[i_rangee*6+i_colonne],cmap='gray')
plt.show()

## Préparation des données

### Conversion des étiquettes-cibles en vecteurs binaires à un bit discriminant

In [ ]:
# Conversion des étiquettes-cibles en vecteurs binaires à un bit discriminant
from keras.utils import to_categorical
classes_cibles_entrainement = to_categorical(classes_cibles_entrainement)
classes_cibles_test = to_categorical(classes_cibles_test)
print("Conversion des étiquettes-cibles en vecteurs binaires terminée!")

### Normalisation des images

In [ ]:
# Normalisation
def normalisation(entrainement, test):
    # convertir de nombres entiers à nombres décimaux
    entrainement_normalise = entrainement.astype('float32')
    test_normalise = test.astype('float32')
    # normalisation à un nombre entre 0 et 1
    entrainement_normalise = entrainement_normalise / 255.0
    test_normalise = test_normalise / 255.0
    return entrainement_normalise, test_normalise

attributs_entrainement, attributs_test = normalisation(attributs_entrainement, attributs_test)

print("Normalisation des images terminée!")

## Construction d'un réseau convolutif - LeNet

<img src="https://courses.edx.org/asset-v1:UMontrealX+Cegep-Matane-VIARENA+2T2024+type@asset+block@LeNet-52.png">

In [ ]:
# Construction du modèle

from keras.models import Sequential
from keras.layers import Conv2D, AveragePooling2D, Flatten, Dense

print("Création d'un modèle LeNet...")

# vecteur / tenseur d'entrée:
#   28 pixels de hauteur,
#   28 pixels de largeur,
#   1 canal de couleur, puisque noir et blanc
dimensions_entree = (28, 28, 1)

# nombre de classes, chiffres de 0 à 9
nombre_classes_cibles = 10

leNet = Sequential()

# Apprentissage et extraction des attributs

# Couche convolutive - C1
#   6 filtres de 5 par 5 pixels
#      5 pixels de hauteur
#      5 pixels de largeur
#   bordure (padding) de 1 pixel, 'same'
#   fonction d'activation tanh
#   incrément de balayage (strides):
#      horizontal = 1
#      vertical = 1
leNet.add(Conv2D(filters=6,
                 kernel_size=(5,5),
                 strides = (1,1),
                 padding = 'same',
                 activation='tanh',
                 input_shape=dimensions_entree))

# Couche sous-échantillonnage (pooling) - S2
#   fenêtre d'échantillonnage de 2 par 2 pixels
#   incrément de balayage (strides):
#      horizontal = 2
#      vertical = 2
leNet.add(AveragePooling2D(pool_size=(2, 2),
                           strides=(2,2)))

# Couche convolutive - C3
#   16 filtres de 5 par 5 pixels
#      5 pixels de hauteur
#      5 pixels de largeur
#   pas de bordure (padding), 'valid'
#   fonction d'activation tanh
#   incrément de balayage (strides):
#      horizontal = 1
#      vertical = 1
leNet.add(Conv2D(filters=16,
                 kernel_size=(5, 5),
                 strides = (1,1),
                 padding = 'valid',
                 activation='tanh'))

# Couche sous-échantillonnage (pooling) - S4
#   fenêtre d'échantillonnage de 2 par 2 pixels
#   incrément de balayage (strides):
#      horizontal = 2
#      vertical = 2
leNet.add(AveragePooling2D(pool_size=(2, 2),
                           strides=(2,2)))

# Classification des représentations par un perceptron multicouche
# aplatissement des représentations en un vecteur unique
leNet.add(Flatten())

# Couche cachée intégralement connectée - C5
leNet.add(Dense(units=120,
                activation='tanh'))
# 2e couche cachée intégralement connectée - F6
leNet.add(Dense(units=84,
                activation='relu'))

# Couche de sortie intégralement connectée
#    nombre de neurones de sortie = nombre_classes_cibles
#    fonction d'activation de sortie = softmax (exponentielle normalisée)
leNet.add(Dense(units=nombre_classes_cibles,
                activation = 'softmax'))

print()
print("Description du modèle de base:")
leNet.summary()

### Compilation du modèle

In [ ]:
print()
print("Compilation du modèle...")

#   comme il s'agit d'une classification multiclasses avec des attributs catégoriels
#   vous allez utiliser categorical_crossentropy
leNet.compile(loss="categorical_crossentropy",
              optimizer="adam",
              metrics=["accuracy"])

print("Modèle compilé!")

# Évaluation du modèle avant entraînement

In [ ]:
# Évaluation du modèle avant entraînement

print()
print("Évaluation du modèle de base avant entraînement...")

resultats = leNet.evaluate(attributs_test, classes_cibles_test, verbose=0)
print("Exactitude test: {:.2f}%".format(resultats[1]*100))

Voila qui est tout à fait normal. Puisque MNIST a 10 étiquettes de classe. Ainsi, en devinant au hasard, nous devrions obtenir une exactitude de 10%.

## Entraînement du modèle

In [ ]:
# Entraînement du modèle

print()
print("Entraînement du modèle...")

batch_size = 128
epochs = 20
taille_jeu_validation = 0.1

# Les images MNIST sont de 28x28 pixels, ce qui est plus petit que ce que LeNet-5
# attend de 32x32 pixels. Une solution simple consiste à utiliser des images de
# 28 par 28 pixels. Une solution plus complexe, serait d'ajouter une bordure de 2 pixels.

# Rappel: attributs_entrainement.shape = (60000, 28, 28) => (60000, 28, 28, 1)

traces_entrainement = leNet.fit(attributs_entrainement,
                                classes_cibles_entrainement,
                                batch_size=batch_size,
                                epochs=epochs,
                                validation_split=taille_jeu_validation)

## Évaluation du modèle avec un jeu de données test

In [ ]:
# Évaluation du modèle

print()
print("Évaluation du modèle...")

# Rappel: attributs_test.shape = (10000, 28, 28) => (10000, 28, 28, 1)

resultats = leNet.evaluate(attributs_test, classes_cibles_test, verbose=0)
print("Exactitude test: {:.2f}%".format(resultats[1]*100))

**Note:** On constate un gain d'environ 5% avec l'utilisation d'un réseau convolutif comme LeNet et l'exactitude obtenue avec un perceptron multicouche. Cette différence démontre l'avantage des réseaux convolutifs pour la reconnaissance d'images même simples comme des chiffres manuscrits en noir et blanc.

## Affichage des courbes d'entraînement

In [ ]:
# Affichage des courbes d'entraînement
hauteur = 6
plt.figure(figsize=(1.618*hauteur,hauteur))
plt.title('Erreur entropie croisée')
plt.plot(traces_entrainement.history['loss'], color='blue', label='courbe entraînement')
plt.plot(traces_entrainement.history['val_loss'], color='orange', label='courbe test')
plt.ylabel("Erreur")
plt.xlabel("Nombre d'époques")
plt.legend()
plt.show()

In [ ]:
# tracer l'exactitude
hauteur = 6
plt.figure(figsize=(1.618*hauteur,hauteur))
plt.title('\nExactitude de la classification')
plt.plot(traces_entrainement.history['accuracy'], color='blue', label='courbe entraînement')
plt.plot(traces_entrainement.history['val_accuracy'], color='orange', label='courbe test')
plt.ylabel("Exactitude")
plt.xlabel("Nombre d'époques")
plt.legend()
plt.show()
# Sauvegarde du graphique en format .png
# nom_graphique = "modele_de_base-courbes_entraînement.png"
# plt.savefig(nom_graphique)
# plt.close()

In [ ]:
print("Exécution carnet IPython terminée!")